## 필요한 라이브러리 불러오기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time, os
from datetime import datetime
import pandas as pd
from selenium.webdriver import ActionChains

## 인기 앱/게임 정보 및 리뷰 크롤링

In [34]:
def pop_crawler(link, app_cnt):
    # 크롬 드라이버(크롬 브라우저 컨트롤) 다운로드 https://sites.google.com/a/chromium.org/chromedriver/home
    driver = webdriver.Chrome('./config/chromedriver')
    driver.get(pop_link)

    #앱 컨테이너
    common_path = '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div'

    # 수집하고자 하는 데이터에 맞게 데이터프레임 생성
    PlaystoreData = pd.DataFrame(columns=['Collection Date', 'App Name', 'App Developer', 'App Category',
                                'App Pop Rank', 'App Download', 'App Review Count', 'App Charge', 'App Limit'])
    PlaystoreReviewData = pd.DataFrame(columns=[
                                      'Collection Date', 'App Name', 'Comment Date', 'User', 'Rating', 'Helpful', 'Comment', 'Answer'])

    # 스크롤 내리는 수 (스크롤 한번 당 리뷰 40개씩 증가), 3의 경우 총 리뷰 120개 수집
    scroll_cnt = 3    
    for i in range(app_cnt):
        if i >= 50:  # 50개 이상의 앱 리뷰를 수집하는 경우 필요합니다. 다만 현재 기준은 최대 100위까지이므로 이후 페이지까지 갱신하고자 하면 코드를 수정해야 합니다.
            #드라이버 새로고침 - 새 화면이 뜰 수 있도록 도움
            driver.refresh()
            time.sleep(1)

            #첫 페이지의 49번째 컨테이너를 선택합니다. 첫 페이지는 무조건 나오므로 첫 페이지의 가장 마지막 요소를 기준으로 합니다.
            move_under = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div[49]')
            action = ActionChains(driver)
            action.move_to_element(move_under).perform()
            time.sleep(1)

            #페이지 확장 시 앱 컨테이너 변경됨
            common_path = '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/c-wiz'
            game_path = driver.find_element_by_xpath(
                common_path+"[{0}]".format(str(i-50+1)))
            game_path.click()
        elif i > 10 and i % 2 == 0:  # 컨테이너 선택에서 발생하는 오류 해결
            driver.refresh()
            time.sleep(1)

            # i번째 컨터이너까지 스크롤 함
            move_under = driver.find_element_by_xpath(common_path+"[{0}]".format(str(i+1)))
            action = ActionChains(driver)
            action.move_to_element(move_under).perform()      

            game_path=driver.find_element_by_xpath(common_path+"[{0}]".format(str(i+1)))
            game_path.click()
        else:
            game_path = driver.find_element_by_xpath(
                common_path+"[{0}]".format(str(i+1)))
            game_path.click()

        os.makedirs('result', exist_ok=True)
        time.sleep(1)

        #앱 정보 가져오기
        Collection_Date = datetime.today().strftime("%Y-%m-%d %H:%M")
        time.sleep(1)
        App_Name = driver.find_element_by_css_selector('.AHFaub > span').text
        App_Rank = i+1

        try:
            App_Review_cnt = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[2]/c-wiz/span/span[1]').text
            App_Developer = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/span[1]/a').text
            App_Category = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/span[2]/a').text

        except:
            App_Review_cnt = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[2]/c-wiz/span/span[1]').text
            App_Developer = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/span[1]').text
            App_Category = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/span[2]/a').text
            pass

        # 맨 아래로 스크롤        
        #해당 앱에서는 more_btn이 다른 위치에 존재함
        if App_Name == '타일 연결 - 무료 타일 퍼즐 & 매치 브레인 게임':
            print('조건문 실행')
            more_btn = driver.find_element_by_xpath(
                "//*[@id='fcxH9b']/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[2]")
            action = ActionChains(driver)
            action.move_to_element(more_btn).perform()
        else:
            more_btn = driver.find_element_by_xpath(
            "//*[@id='fcxH9b']/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[2]")
            action = ActionChains(driver)
            action.move_to_element(more_btn).perform()

        # 하단 앱 정보 가져오기
        try:
            App_Download = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[3]/span/div/span').text
            App_Limit = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[6]/span/div/span/div[1]').text
        except:
            App_Download = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[1]/div[2]/div/div[3]/span/div/span').text
            App_Limit = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[1]/div[2]/div/div[6]/span/div/span/div[1]').text
            pass

        App_Charge = 'Free'

        # 앱 정보 데이터프레임에 추가
        PlaystoreData = PlaystoreData.append({
            'Collection Date': Collection_Date,
            'App Name': App_Name,
            'App Developer': App_Developer,
            'App Category': App_Category,
            'App Pop Rank': App_Rank,
            'App Download': App_Download,
            'App Review Count': App_Review_cnt,
            'App Charge': App_Charge,
            'App Limit': App_Limit
        }, ignore_index=True)
        driver.find_element_by_xpath(
            '//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span').click()

        for i in range(scroll_cnt):
            time.sleep(2)
            scroll = driver.find_element_by_xpath(
                '//*[@id="fcxH9b"]/div[4]/c-wiz[3]/div/div[2]/div/div/main/div/div[2]')  # 스크롤해야 할 위치 찾기
            action = ActionChains(driver)
            action.move_to_element(scroll).perform()  # 스크롤 실행

        print(App_Name, '의 데이터를 수집하고 있습니다')
        # 리뷰 컨테이너 가져오기
        reviews = driver.find_elements_by_xpath(
            '//*[@jsname="fk8dgd"]//div[@class="d15Mdf bAhLNe"]')
        print(' %d 개의 리뷰가 존재합니다.' % len(reviews))

        # 리뷰 데이터 가져오기
        for review in reviews:
            #BeautifulSoup 객체 가져오기
            soup = BeautifulSoup(review.get_attribute('innerHTML'), 'html.parser')

            # 리뷰한 유저 이름
            user = soup.find(class_='X43Kjb').text

            # 별점
            rating = int(soup.find('div', role='img').get(
                'aria-label').replace('별표 5개 만점에', '').replace('개를 받았습니다.', '').strip())

            # 리뷰 날짜
            comment_date = soup.find(class_='p2TkOb').text
            comment_date = datetime.strptime(comment_date, '%Y년 %m월 %d일')
            comment_date = comment_date.strftime('%Y-%m-%d')

            # 유용함
            helpful = soup.find(class_='jUL89d y92BAb').text
            if not helpful:
                helpful = 0

            # 리뷰 내역
            comment = soup.find('span', jsname='fbQN7e').text
            if not comment:
                comment = soup.find('span', jsname='bN97Pc').text

            # 리뷰에 대한 개발사 답변
            answer = None
            a_div = soup.find('div', class_='LVQB0b')
            if a_div:
                answer = a_div.text.replace('\n', ' ')

            # 리뷰 데이터 데이터프레임에 추가
            PlaystoreReviewData = PlaystoreReviewData.append({
                'Collection Date': Collection_Date,
                'App Name': App_Name,
                'Comment Date': comment_date,
                'User': user,
                'Rating': rating,
                'Helpful': helpful,
                'Comment': comment,
                'Answer': answer
            }, ignore_index=True)

        print(App_Name, '은(는) ', App_Rank, '위입니다. 이 앱은 리뷰 수집이 끝났습니다.')

        driver.back()
        driver.back()


    driver.stop_client()
    driver.close()
    return PlaystoreData, PlaystoreReviewData


## 최고 매출 게임 정보 및 리뷰 크롤링

In [42]:
def best_crawler(link, app_cnt):
    # 크롬 드라이버(크롬 브라우저 컨트롤) 다운로드 https://sites.google.com/a/chromium.org/chromedriver/home
    driver = webdriver.Chrome('./config/chromedriver')
    driver.get(best_link)

    #앱 컨테이너
    common_path = '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div'                 
    # 수집하고자 하는 데이터에 맞게 데이터프레임 생성
    PlaystoreBest = pd.DataFrame(columns=['Collection Date', 'App Name', 'App Developer', 'App Category', 'App Best Rank', 'App Download', 'App Review Count', 'App Charge', 'App Limit'])
    
    for i in range(app_cnt):
        if i >= 50: #50개 이상의 앱 리뷰를 수집하는 경우 필요합니다. 다만 현재 기준은 최대 100위까지이므로 이후 페이지까지 갱신하고자 하면 코드를 수정해야 합니다.
            #드라이버 새로고침 - 새 화면이 뜰 수 있도록 도움
            driver.refresh()
            time.sleep(1)
            
            move_under = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div[49]')
            action = ActionChains(driver)
            action.move_to_element(move_under).perform()
            time.sleep(1)

            #페이지 확장 시 앱 컨테이너 변경됨
            common_path = '//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/c-wiz'
            game_path=driver.find_element_by_xpath(common_path+"[{0}]".format(str(i-50+1)))
            game_path.click()
        elif i % 16 == 0 or i % 28 == 0:
            driver.refresh()
            time.sleep(1)
            move_under = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/c-wiz/div/c-wiz/c-wiz/c-wiz/div/div[2]/div[{0}]'.format(str(i+1)))
            action = ActionChains(driver)
            action.move_to_element(move_under).perform()      
            game_path=driver.find_element_by_xpath(common_path+"[{0}]".format(str(i+1)))
            game_path.click()
        else:
            game_path=driver.find_element_by_xpath(common_path+"[{0}]".format(str(i+1)))
            game_path.click()
        
        os.makedirs('result', exist_ok=True) 
        time.sleep(1)
        
        #앱 정보 가져오기
        Collection_Date = datetime.today().strftime("%Y-%m-%d %H:%M")
        time.sleep(1)
        App_Name = driver.find_element_by_css_selector('.AHFaub > span').text
        App_Rank = i+1

        print(App_Name,'의 데이터를 수집하고 있습니다')  
        
        try:
            App_Review_cnt = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[2]/c-wiz/span/span[1]').text
            App_Developer = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/span[1]/a').text
            App_Category = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/span[2]/a').text

        except:
            App_Review_cnt = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[2]/c-wiz/span/span[1]').text
            App_Developer = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/span[1]').text
            App_Category = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[1]/c-wiz[1]/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/span[2]/a').text
            pass

        # 맨 아래로 스크롤
        more_btn = driver.find_element_by_xpath("//*[@id='fcxH9b']/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[2]")
        action = ActionChains(driver)
        action.move_to_element(more_btn).perform()
        # 하단 앱 정보 가져오기
        try:
            App_Download = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[3]/span/div/span').text
        except:
            App_Download = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[1]/div[2]/div/div[3]/span/div/span').text
            pass
        
        try:
            App_Limit = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[6]/span/div/span/div[1]').text
        except:
            try:
                App_Limit = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[3]/div[1]/div[2]/div/div[7]/span/div/span/div[1]').text
            except:
                App_Limit = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/c-wiz[2]/div[1]/div[2]/div/div[6]/span/div/span/div[1]').text
                pass
            pass
            
        # 앱 정보 데이터프레임에 추가
        PlaystoreBest = PlaystoreBest.append({
            'Collection Date': Collection_Date,
            'App Name': App_Name,
            'App Developer': App_Developer,
            'App Category': App_Category,
            'App Best Rank': App_Rank,
            'App Download': App_Download,
            'App Review Count': App_Review_cnt,
            'App Limit': App_Limit
        }, ignore_index=True)
      
        print(App_Name,'은(는) ', App_Rank, '위입니다. 앱 정보 수집을 마쳤습니다.')
      
        driver.back()

    driver.stop_client()
    driver.close()
    return PlaystoreBest

## 메인 코드

In [35]:
if __name__ == "__main__":
  # 수집하고자 하는 앱 개수 (인기 앱/게임의 경우 한 페이지당 최대 50위까지 가능)
  app_cnt = 1

  # 인기 앱/게임 페이지 링크
  pop_link = 'https://play.google.com/store/apps/collection/cluster?clp=0g4cChoKFHRvcHNlbGxpbmdfZnJlZV9HQU1FEAcYAw%3D%3D:S:ANO1ljJ_Y5U&gsr=Ch_SDhwKGgoUdG9wc2VsbGluZ19mcmVlX0dBTUUQBxgD:S:ANO1ljL4b8c'
  PlaystoreData, PlaystoreReviewData = pop_crawler(pop_link, app_cnt)

  # 최고 매출 게임 페이지 링크
  # best_link = 'https://play.google.com/store/apps/collection/cluster?clp=0g4YChYKEHRvcGdyb3NzaW5nX0dBTUUQBxgD:S:ANO1ljLhYwQ&gsr=ChvSDhgKFgoQdG9wZ3Jvc3NpbmdfR0FNRRAHGAM%3D:S:ANO1ljIKta8'
  # PlaystoreBest = best_crawler(best_link, app_cnt)

오딘: 발할라 라이징 의 데이터를 수집하고 있습니다
 120 개의 리뷰가 존재합니다.
오딘: 발할라 라이징 은(는)  1 위입니다. 이 앱은 리뷰 수집이 끝났습니다.


In [ ]:
PlaystoreData

In [20]:
PlaystoreReviewData

,Collection Date,App Name,Comment Date,User,Rating,Helpful,Comment,Answer
0,2021-07-05,오딘: 발할라 라이징,2021-07-02,안유민,1,162,첫날에 결제를 하고 꾸준히 소소하게 즐기려고 했으나 대기열이 많아 접속자체를 할수가...,Kakao Games Corp.2021년 7월 5일안녕하세요. 신의 영역을 차지하라...
1,2021-07-05,오딘: 발할라 라이징,2021-07-02,어제보다오늘더c.k. C,1,132,모바일 게임 그동안 참 많이 했는데... 다른 게임들은 그나마 실행후 플레이 하면서...,None
2,2021-07-05,오딘: 발할라 라이징,2021-06-29,유승엽,3,65,제2랑 차이보면 카툰과 실사느낌차이. 캐릭터 커스터마이징 비슷함. 이직 초반이지만 ...,None
3,2021-07-05,오딘: 발할라 라이징,2021-07-01,김용욱,1,58,자 자 어떤게임인지 간단히 소개해드릴께요. 전사 궁수 법사 힐러 고르세요. 화려한그...,None
4,2021-07-05,오딘: 발할라 라이징,2021-06-30,노랑꼬리,1,33,문장을 구매해서 강화하는데 무조건 실패시 파괴 ㅋㅋㅋㅋㅋㅋ 파괴 당하고 바로 삭제 ...,None
...,...,...,...,...,...,...,...,...
10523,2021-07-05,Rider,2021-06-09,Hyunjo Kim,2,2,차고로 들어가면 게임이 멈춤,None
10524,2021-07-05,Rider,2021-06-13,Mr김치,1,0,응 개똥게임 차선택이안된고클릭도 안되,None
10525,2021-07-05,Rider,2021-06-25,박점숙,5,0,눈동자 지 너무 잘 먹고,None
10526,2021-07-05,Rider,2021-06-15,장문자,5,0,그냥그대로멈추는데? 와 버그대박 불상해서 많ㅇㄱ드릴게요,None


## 데이터셋 저장하기

In [14]:
# 데이터셋, 파일이름 파라미터로 받음
def make_dataset(data, filename): 
    data.to_csv('./data/'+filename+'.csv', encoding='utf-8-sig', index=False)
    data.to_json('./data/'+filename+'.json', orient='table')


In [21]:
# 데이터셋과 이름 설정
make_dataset(PlaystoreBest, 'PlaystoreBest')
make_dataset(PlaystoreReviewData, 'PlaystoreReviewData')
make_dataset(PlaystoreData, 'PlaystoreData')